In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

In [2]:
king_county = pd.read_csv('./data/EXTR_RPSale.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_KC = pd.DataFrame(king_county)

In [4]:
df_KC.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,2324275,142205,9172,12/05/2007,650000,20071212001568,,,,,,,COLE VERN+JITKA ...,MINTON DAVID W+KIMBERLY A ...,3,6,3,N,N,N,N,1,8,
1,952729,278470,10,07/10/1987,0,198707130302,067,051,730140,P,"101-108,",,SEAFIRST MTG CORP ...,SEATTLE FIRST NATIONAL BANK ...,3,0,15,N,N,N,N,18,3,45
2,952729,278470,40,07/10/1987,0,198707130302,067,051,730140,P,"101-108,",,SEAFIRST MTG CORP ...,SEATTLE FIRST NATIONAL BANK ...,3,0,15,N,N,N,N,18,3,45
3,952729,278470,100,07/10/1987,0,198707130302,067,051,730140,P,"101-108,",,SEAFIRST MTG CORP ...,SEATTLE FIRST NATIONAL BANK ...,3,0,15,N,N,N,N,18,3,45
4,952729,278470,190,07/10/1987,0,198707130302,067,051,730140,P,"101-108,",,SEAFIRST MTG CORP ...,SEATTLE FIRST NATIONAL BANK ...,3,0,15,N,N,N,N,18,3,45


In [5]:
sales_price_dict = dict(df_KC.SalePrice.value_counts())
total = 0
for key, val in sales_price_dict.items():
    if key != 0:
        total += val
print("Total number of Sales Price > $0 = {}".format(total))
print("Total number of Sales Price = $0 = {}".format(sales_price_dict[0]))
print("Percentage of missing values = {}".format(sales_price_dict[0]/total))

Total number of Sales Price > $0 = 1337800
Total number of Sales Price = $0 = 706282
Percentage of missing values = 0.5279428913140978


In [ ]:
df_KC.info()

In [ ]:
df_KC.isnull().sum()

In [6]:
kc_columns = list(df_KC.columns)
str_columns = ['DocumentDate', 'PlatType', 'PlatLot', 'SellerName', 'BuyerName', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse', 'AFHistoricProperty']
[kc_columns.remove(x) for x in str_columns]
kc_int_columns = kc_columns
kc_str_columns = ['PlatType', 'PlatLot', 'SellerName', 'BuyerName', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse', 'AFHistoricProperty']

In [ ]:
kc_int_columns

In [ ]:
kc_str_columns

In [7]:
for column in kc_int_columns:
    df_KC[column] = pd.to_numeric(column, errors='coerce')

In [ ]:
df_KC.DocumentDate = [pd.to_datetime(x, format='%m%d%Y', errors='coerce') for x in df_KC.DocumentDate]

In [ ]:
for column in kc_str_columns:
    print(column)
    df_KC[column] = df_KC[column].astype(str)
    
# df_KC = df_KC.to_string(columns = kc_str_columns)
# total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)

In [ ]:
correlation = df.corr()
plt.figure(figsize=(14,12))
heatmap = sns.heatmap(correlation, annot=True, linewidths=0, vmin=-1, cmap="RdBu_r")